In [24]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
parasharmanas_movie_recommendation_system_path = kagglehub.dataset_download('parasharmanas/movie-recommendation-system')

print('Data source import complete.')


Using Colab cache for faster access to the 'movie-recommendation-system' dataset.
Data source import complete.


In [25]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px
import warnings
import plotly.io as pio
pio.renderers.default = "iframe_connected"

warnings.filterwarnings("ignore")
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/movie-recommendation-system/movies.csv
/kaggle/input/movie-recommendation-system/ratings.csv


In [26]:
movies = pd.read_csv('movies.csv')
rating = pd.read_csv('ratings.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [27]:
movies.shape

(62423, 3)

In [28]:
rating.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1.147880e+09
1,1,306,3.5,1.147869e+09
2,1,307,5.0,1.147869e+09
3,1,665,5.0,1.147879e+09
4,1,899,3.5,1.147869e+09


In [29]:
rating.shape

(6668919, 4)

In [30]:
data = pd.merge(movies, rating, on='movieId', how='left')

In [31]:
data.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2.0,3.5,1.141416e+09
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1.439472e+09
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,3.0,1.573944e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.586259e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,8.904925e+08


In [32]:
data.nunique()

,0
movieId,62423
title,62325
genres,1639
userId,43147
rating,10
timestamp,5466702


# One hot encoding the genres for better features


In [33]:
genre_dummies = data['genres'].str.get_dummies('|')
data = pd.concat([data, genre_dummies], axis=1)

In [34]:
data.head(100)

,movieId,title,genres,userId,rating,timestamp,(no genres listed),Action,Adventure,Animation,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2.0,3.5,1.141416e+09,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1.439472e+09,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,3.0,1.573944e+09,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.586259e+08,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,8.904925e+08,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,329.0,5.0,8.357078e+08,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
96,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,339.0,3.5,1.436842e+09,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
97,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,340.0,4.0,8.691567e+08,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
98,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,341.0,3.0,8.332817e+08,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [35]:
data.columns

Index(['movieId', 'title', 'genres', 'userId', 'rating', 'timestamp',
       '(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
       'War', 'Western'],
      dtype='object')

# **RECOMMENDATION OBJECTIVE:**

We are going to give top 5 movies which the user would like to watch, so this is our objective.

In this case let's assume we have an OTT platform which has 100 movies and our user X has watched only 5 movies in that, So the number of unwatched movies is 95. And here our algorithim should give top 5 movies form the 95 unwatched movies.


Great we have defined our problem, Now how are we going to do this...
There are many approaches for recommendation system, here we will concendrate on Collaborative filtering.

In order to explain this alogorithm, let me give you an example

Let us consider two users
User A, User B

User A has watched Movie_1, Movie_2 and Movie_3.
User B has watched Movie_1, Movie_2.

So User A and User B has watched both Movie_1 and Movie_2, they have common choice and taste.
Hence there is a high probability that User B would like to watch Movie_3 which was watched by User A.This is the core idea behind collaborative filtering.

Yep, now We have the dataset having list of movies, users, genre ect.
In order to do collaborative filtering we are going to compute the similarity between the users.
We are going to convert each user and the movies watched by them into vector format(numerical representation) and calculate the distance between each numerical representation of users.
The nearest vectors to a user X(vector representation) are the users who have common interest with that particular user X.


Donee!!!!

As we have cosine similarity,
The next step we will be finding out the list of similar users
So we give 12244.0 as user_id and its going to give the top list of 5 users who are having similar taste like to that user.

So yeah we have the list of similar users now!

How to recommend movies ?

We are going to predict what the user will rate for unseen movies

We have the similar users data now, who are them, what movies they have watched, what rating they have given, we have all this.

So lets assume Our user X has not watched 'Tenet' but the similar users have watched 'Tenet' and also rated it.
So we are going to just take those ratings given by those similar users and average it(average in the sense we will be using weights and other techniques to compute what our user will rate 'Tenet'
based on the ratings given by the similar users).

So yeah this is the core.

And now once we predicted the ratings for all the unseen movies we are going to sort it based on high ratings and the top 5 rated movies will be recommended to the user.

Yep

I have just added this explantion as a part of my learning, call me out if I got anything off.


**We are just going to take popular movies, users(who rated high number of movies) for computational efficiency**

In [36]:
# Filter top users and movies
top_users = data['userId'].value_counts().head(500).index
top_movies = data['movieId'].value_counts().head(1000).index

sample_data = data[data['userId'].isin(top_users) & data['movieId'].isin(top_movies)]

# Now pivot safely
user_movie_matrix = sample_data.pivot_table(index='userId', columns='movieId', values='rating')


# **Cosine Similarity**

In [37]:
user_sim = cosine_similarity(user_movie_matrix.fillna(0))
user_sim_df = pd.DataFrame(user_sim, index=user_movie_matrix.index, columns=user_movie_matrix.index)


In [38]:
def get_top_similar_users(target_user, n=5):
    similar_users = (
        user_sim_df[target_user]
        .sort_values(ascending=False)
        .drop(target_user)  # remove self
        .head(n)
    )
    return similar_users


# **List of similar users for user_id 12244.0**

In [39]:
# Get a valid user ID from the matrix (e.g., the first one available)
valid_user_id = user_sim_df.index[0]
print(f"Finding similar users for User ID: {valid_user_id}")
get_top_similar_users(valid_user_id)

Finding similar users for User ID: 187.0


,187.0
userId,
30391.0,0.706712
38588.0,0.704971
20302.0,0.703344
29104.0,0.701887
29898.0,0.701790


In [40]:
def predict_rating(user_id, movie_title, k=5):
    similar_users = get_top_similar_users(user_id, k)

    sim_scores = []
    weighted_ratings = []

    for sim_user, sim_score in similar_users.items():
        rating = user_movie_matrix.loc[sim_user, movie_title]
        if not np.isnan(rating):
            sim_scores.append(sim_score)
            weighted_ratings.append(sim_score * rating)

    if len(sim_scores) == 0:
        return np.nan

    return np.sum(weighted_ratings) / np.sum(sim_scores)


In [41]:
def recommend_movies(user_id, n=5):
    user_ratings = user_movie_matrix.loc[user_id]
    unseen_movies = user_ratings[user_ratings.isna()].index

    predictions = {}
    for movie in unseen_movies:
        rating_pred = predict_rating(user_id, movie)
        if not np.isnan(rating_pred):
            predictions[movie] = rating_pred

    top_movies = sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:n]

    recommendations = []
    for movie, rating in top_movies:
        title = data.loc[data['movieId'] == movie, 'title'].values
        title = title[0] if len(title) > 0 else "Unknown Title"
        recommendations.append((title, rating))

    return recommendations


# **Final Recommendation for user_id 12244.0**

In [42]:
recommend_movies(12244.0, n=5)

[('2001: A Space Odyssey (1968)', np.float64(5.0)),
 ('Great Escape, The (1963)', np.float64(5.0)),
 ('French Connection, The (1971)', np.float64(5.0)),
 ('Avengers: Infinity War - Part I (2018)', np.float64(5.0)),
 ('Logan (2017)', np.float64(4.835799110031907))]

In [43]:
def visualize_recommendations(user_id, n=5):
    recommendations = recommend_movies(user_id, n)
    if not recommendations:
        print("No recommendations available.")
        return

    df = pd.DataFrame(recommendations, columns=['Movie', 'Predicted Rating'])

    # Create horizontal bar chart
    fig = px.bar(
        df,
        x='Predicted Rating',
        y='Movie',
        orientation='h',
        text='Predicted Rating',
        color='Predicted Rating',
        color_continuous_scale='Viridis'
    )

    fig.update_layout(
        title=f"Top {n} Movie Recommendations for User {user_id}",
        xaxis_title="Predicted Rating",
        yaxis_title="Movie",
        yaxis={'categoryorder':'total ascending'},
        plot_bgcolor='rgba(245,245,245,1)',
        font=dict(family="Arial", size=14)
    )

    fig.update_traces(texttemplate='%{text:.2f}', textposition='inside', marker_line_color='black', marker_line_width=1.5)

    fig.show()


In [47]:
import plotly.io as pio
pio.renderers.default = 'colab'

# Using the valid_user_id defined in the previous cell to ensure it works
visualize_recommendations(valid_user_id, n=5)